***Simulação da Fila Servidores Sem Prioriedade***

In [ ]:
pip install simpy

In [40]:
#Bibliotecas utilizadas
import simpy
import random

In [41]:
#Variaveis
tempo_simulacao = 60                #tempo em minutos da simulação
n_servidores = 10                    #numero de servidores comuns
n_servidores_pri = 0                #numero de servidores que atendem a fila priorietária
taxa_chegada = 1                    #tempo de chegada de novos clientes
tempo_c = 9                         #tempo médio de atendimento para cliente sem prioriedade
tempo_p = 12                        #tempo médio de atendimento para cliente com prioriedade
qtd_chamadas_c = 0                  #Quantidade de chamadas finalizadas de clientes comuns
qtd_chamadas_p = 0                  #Quantidade de chamadas finalizadas de clientes priorietários
prc_priorietario = 0.3              #porcentagem dos clientes priorietários
tempo_espera_P = []
tempo_espera_C = []

In [42]:
#Função do processo de atendimento/fila
def cliente(env, nome, tipo, atendentes):
  chegada = env.now
  print(f'{nome} chegou às {chegada:0.2f} minutos')

  with atendentes.request() as req:
    yield req
    espera = env.now - chegada
    print(f'{nome} ({tipo})começou atendimento às {env.now: .2f} (esperou {espera: .2f} minutos)')

    #Duração do atendimento seguindo uma variancia conforme o tempo médio de atendimento
    if tipo == 'P':
      duracao = random.expovariate(1.0 / tempo_p)
      tempo_espera_P.append(espera)
    else:
      duracao = random.expovariate(1.0 / tempo_c)
      tempo_espera_C.append(espera)

    yield env.timeout(duracao)
    print(f'{nome} ({tipo})terminou atendimento às {env.now: .2f})')
    if tipo == 'P':
      global qtd_chamadas_p
      qtd_chamadas_p += 1
    else:
      global qtd_chamadas_c
      qtd_chamadas_c += 1


#Geração de clinetes
def gera_clientes(env, atendentes):
  i = 0
  while True:
    yield env.timeout(random.expovariate(taxa_chegada))#tempo entre chegadas
    i += 1
    tipo = 'P' if random.random() < prc_priorietario else 'C'
    env.process(cliente(env, f'Cliente {i}', tipo, atendentes))

In [ ]:
#Ambiente de simulação da fila comum sem servidores priorietários
env = simpy.Environment()
atendentes = simpy.Resource(env, capacity=n_servidores)
env.process(gera_clientes(env, atendentes))
env.run(until=tempo_simulacao)

In [ ]:
print(f'Quantidade de chamadas com prioridade: {qtd_chamadas_p}')
print(f'Quantidade de chamadas sem prioridade: {qtd_chamadas_c}')

media_espera_C = sum(tempo_espera_C) / len(tempo_espera_C)
print(f"\nTempo médio de espera fila comum: {media_espera_C:.2f} minutos")

media_espera_P = sum(tempo_espera_P) / len(tempo_espera_P)
print(f"\nTempo médio de espera fila Priorietária: {media_espera_P:.2f} minutos")

***Simulação da Fila com Servidores Prioritários***

In [ ]:
#Variaveis
tempo_simulacao = 60                #tempo em minutos da simulação
n_servidores = 6                    #numero de servidores comuns
n_servidores_pri = 4                #numero de servidores que atendem a fila priorietária
taxa_chegada = 1                    #tempo de chegada de novos clientes
tempo_c = 9                         #tempo médio de atendimento para cliente sem prioriedade
tempo_p = 12                        #tempo médio de atendimento para cliente com prioriedade
qtd_chamadas_c = 0                  #Quantidade de chamadas finalizadas de clientes comuns
qtd_chamadas_p = 0                  #Quantidade de chamadas finalizadas de clientes priorietários
prc_priorietario = 0.3              #porcentagem dos clientes priorietários
tempo_espera_P = []
tempo_espera_C = []

In [ ]:
#Função do processo de atendimento/fila
def cliente(env, nome, tipo, atendentes_p, atendentes_c):
  chegada = env.now
  print(f'{nome} chegou às {chegada:0.2f} minutos')

  if tipo == 'P':
    with atendentes_p.request() as req:
      yield req
      espera = env.now - chegada
      tempo_espera_P.append(espera)
      print(f'{nome} ({tipo})começou atendimento às {env.now: .2f} (esperou {espera: .2f} minutos)')
      duracao = random.expovariate(1.0 / tempo_p)
      yield env.timeout(duracao)
      print(f"{nome} ({tipo}) terminou atendimento às {env.now:.2f} min")

      global qtd_chamadas_p
      qtd_chamadas_p += 1

  else:
    with atendentes_c.request() as req:
      yield req
      espera = env.now - chegada
      tempo_espera_C.append(espera)
      print(f'{nome} ({tipo})começou atendimento às {env.now: .2f} (esperou {espera: .2f} minutos)')
      duracao = random.expovariate(1.0 / tempo_c)
      yield env.timeout(duracao)
      print(f"{nome} ({tipo}) terminou atendimento às {env.now:.2f} min")

      global qtd_chamadas_c
      qtd_chamadas_c += 1


#Geração de clinetes
def gera_clientes(env, atendentes_p, atendentes_c):
  i = 0
  while True:
    yield env.timeout(random.expovariate(taxa_chegada))#tempo entre chegadas
    i += 1
    tipo = 'P' if random.random() < prc_priorietario else 'C'
    env.process(cliente(env, f'Cliente {i}', tipo, atendentes_p, atendentes_c))

In [ ]:
#Ambiente de simulação da fila com servidores priorietários
env = simpy.Environment()
atendentes_p = simpy.Resource(env, capacity=n_servidores_pri)
atendentes_c = simpy.Resource(env, capacity=n_servidores)
env.process(gera_clientes(env, atendentes_p, atendentes_c))
env.run(until=tempo_simulacao)

In [ ]:
print(f'Quantidade de chamadas com prioridade: {qtd_chamadas_p}')
print(f'Quantidade de chamadas sem prioridade: {qtd_chamadas_c}')

media_espera_C = sum(tempo_espera_C) / len(tempo_espera_C)
print(f"\nTempo médio de espera fila comum: {media_espera_C:.2f} minutos")

media_espera_P = sum(tempo_espera_P) / len(tempo_espera_P)
print(f"\nTempo médio de espera fila Priorietária: {media_espera_P:.2f} minutos")